In [ ]:
import os
import numpy as np
from torch.utils.data import Dataset
import gzip
import matplotlib.pyplot as plt  # 绘图库


class Mnist(Dataset):
    def __init__(self, root, train=True, transform=None):
        # 根据是否为训练集，得到文件名前缀
        self.file_pre = 'train' if train == True else 't10k'
        self.transform = transform  # 定义变换函数
        # 生成对应数据集的图片和标签文件路径
        self.label_path = os.path.join(root,
                                       '%s-labels-idx1-ubyte.gz' % self.file_pre)
        self.image_path = os.path.join(root,
                                       '%s-images-idx3-ubyte.gz' % self.file_pre)

        # 读取文件数据，返回图片和标签
        self.images, self.labels = self.__read_data__(
            self.image_path,
            self.label_path)

    def __read_data__(self, image_path, label_path):
        # 数据集读取
        with gzip.open(label_path, 'rb') as lbpath:
            labels = np.frombuffer(lbpath.read(), np.uint8,
                                   offset=8)  # 将data以流的形式读入转化成ndarray对象，ndarray对象是用于存放同类型元素的多维数组
        with gzip.open(image_path, 'rb') as imgpath:
            images = np.frombuffer(imgpath.read(), np.uint8,
                                   offset=16).reshape(len(labels), 28, 28)  # 将图片以标签文件的元素个数读取，设置大小为28*28
        return images, labels

    def __getitem__(self, index):  # 迭代使用？使用Minist()会调用_getitem_
        image, label = self.images[index], int(self.labels[index])

        # 如果需要转成 tensor，（RGB,HWC）张量， 则使用 tansform
        if self.transform is not None:
            image = self.transform(np.array(image))  # 此处需要用 np.array(image)，转化为数组
        return image, label

    def __len__(self):  # 获取元素个数
        return len(self.labels)


if __name__ == '__main__':
    # 生成实例
    train_set = Mnist(
        root=r'C:\Users\Admin\Desktop\projects\Pytorch_study\mnist识别\mnist',  # MNIST数据集路径
        #train=False,
        #"C:\Users\Admin\Desktop\projects\Pytorch_study\mnist识别\mnist\train-images.idx3-ubyte"
    )
    # 取一组数据并展示
    (data, label) = train_set[0]  # 第一组数据
    plt.imshow(data.reshape(28, 28), cmap='gray')  # 灰色图像
    plt.title('label is :{}'.format(label))
    plt.show()


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class LeNet5(nn.Module):
    def __init__(self):  # 初始化函数
        super(LeNet5, self).__init__()  # 多基层一般使用super
        self.conv1 = nn.Conv2d(1, 6, 5)  # 定义第一个卷积层，1是通道数灰度图片，6是卷积核个数，5卷积核大小
        self.pool1 = nn.MaxPool2d(2, 2)  # 池化核大小2*2，步距也为2,池化层，只改变高和宽，深度不改变
        self.conv2 = nn.Conv2d(6, 16, 5)  # 输入变为6，因为通过第一个卷积层有6个卷积核，输出深度为6
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 展开成一维的，第一层120个节点，输入16*4*4个节点
        self.fc2 = nn.Linear(120, 84)  # 输入120，设置84个节点
        self.fc3 = nn.Linear(84, 10)  # 输出根据训练集修改

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))  # input(1,28,28),output1(6,24,24) output2(6,12,12)
        x = self.pool2(F.relu(self.conv2(x)))  # input(6,12,12),output1(16,8,8) output2(16,4,4)
        x = x.view(-1, 16 * 4 * 4)  # -1第一个维度
        x = F.relu(self.fc1(x))  # 全连接层1及其激活函数
        x = F.relu(self.fc2(x))  # 全连接层3得到输出
        x = self.fc3(x)
        return x


if __name__ == '__main__':
    net = LeNet5()
    print(net)


In [ ]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim#优化器
from torch.utils.data import DataLoader
from data import Mnist
from model import LeNet5

# 生成训练集
train_set = Mnist(
    root=r'C:\\Users\\Admin\\Desktop\\projects\\Pytorch_study\\mnist识别\\mnist',
    train=True,#训练集
    transform=transforms.Compose([#Compose方法是将多种变换组合在一起。
        transforms.ToTensor(),#函数接受PIL Image或numpy.ndarray，将其先由HWC转置为CHW格式
        transforms.Normalize((0.1037,), (0.3081,))#灰度图像，一个通道，均值和方差，标准化
    ])
)
train_loader = DataLoader(#主要用来将自定义的数据读取接口的输出或者PyTorch已有的数据读取接口的输入按照batch size封装成Tensor
    dataset=train_set,#输出的数据
    batch_size=32,
    shuffle=True#将元素随机排序
)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#判断能否调用GPU
print(device)
# 实例化一个网络
net = LeNet5().to(device) #将网络放进GPU

# 定义损失函数和优化器
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(
    net.parameters(),#网络参数
    lr=0.001,#学习率
    momentum=0.9#Momentum 用于加速 SGD（随机梯度下降）在某一方向上的搜索以及抑制震荡的发生。
)

# 3 训练模型
loss_list = []#保存损失函数的值
for epoch in range(10):#训练10次
    running_loss = 0.0#误差清零？
    for batch_idx, data in enumerate(train_loader, start=0):#enumerate索引函数，start下标开始位置

        images, labels = data                       # 读取一个batch的数据
        images=images.to(device)  #将images放进GPU
        labels=labels.to(device)  #将labels放进GPU
        optimizer.zero_grad()                       # 梯度清零，初始化,如果不初始化，则梯度会叠加
        outputs = net(images)                      # 前向传播
        loss = loss_function(outputs, labels)       # 计算误差，label标准？
        loss.backward()                             # 反向传播
        optimizer.step()                            # 权重更新
        running_loss += loss.item()                 # 误差累计

        # 每300个batch 打印一次损失值
        if batch_idx % 300 == 299:#（0-299）（300-599）
            print('epoch:{} batch_idx:{} loss:{}'
                  .format(epoch+1, batch_idx+1, running_loss/300))
            loss_list.append(running_loss/300)#将新的每个平均误差加到损失函数列表后面
            running_loss = 0.0                  #误差清零
print('Finished Training.')

torch.save(net.state_dict(),"Linear.pth")#保存训练模型

# 打印损失值变化曲线
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.title('traning loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

# 测试
test_set = Mnist(#生成测试集
    root='C:\\Users\\Admin\\Desktop\\projects\\Pytorch_study\\mnist识别\\mnist',
    train=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1037,), (0.3081,))
    ])
)
test_loader = DataLoader(
    dataset=test_set,
    batch_size=32,
    shuffle=True
)
correct = 0  # 预测正确数
total = 0    # 总图片数
for data in test_loader:
    images, labels = data
    images=images.to(device)
    labels=labels.to(device)
    outputs = net(images)
    _, predict = torch.max(outputs.data, 1)#1是指按行，0是按列，-1是指最后一个维度，一般也是按行
    total += labels.size(0)
    correct += (predict == labels).sum()

print('测试集准确率 {}%'.format(100*correct // total))

#检测网络
test_output=net(images[:10])#在测试集中选择10张图片输入网络，得到结果
pred_y = torch.max(test_output, 1)[1].data#对得到的结果进行预测
print(pred_y, 'prediction numbe')#输出预测结果
print(labels[:10], 'real number')#输出真实结果


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from data import Mnist
from model import LeNet5
import matplotlib.pyplot as plt

net = LeNet5()
net.load_state_dict(torch.load("Linear.pth"))  # 调用训练好的网络

# 检测网络
test_set = Mnist(  # 生成测试集
    root='C:\\Users\\Admin\\Desktop\\projects\\Pytorch_study\\mnist识别\\mnist',
    train=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1037,), (0.3081,))
    ])
)
test_loader = DataLoader(
    dataset=test_set,
    batch_size=32,
    shuffle=True
)

for data in test_loader:
    images, labels = data

test_output = net(images[:10])  # 在测试集中选择10张图片输入网络，得到结果
pred_y = torch.max(test_output, 1)[1].data  # 对得到的结果进行预测
print(pred_y, 'prediction numbe')  # 输出预测结果
print(labels[:10], 'real number')  # 输出真实结果


def plt_image(image):  # 定义一个函数，将需要预测的手写数字图画出来
    n = 10
    plt.figure(figsize=(10, 4))
    for i in range(n):
        ax = plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()


plt_image(images)

# 测试自己手动设计的手写数字
from PIL import Image

I = Image.open('2.jpg')
L = I.convert('L')  # 转化为二值图像
plt.imshow(L, cmap='gray')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1037,), (0.3081,))
])
im = transform(L)  # [C, H, W]
im = torch.unsqueeze(im, dim=0)  # [N, C, H, W]，扩展维度
with torch.no_grad():
    outputs = net(im)
    _, predict = torch.max(outputs.data, 1)
    print(predict)
